In [1]:
import pandas as pd
from scipy.sparse import csr_matrix, diags
import numpy as np
from sklearn.metrics.cluster import normalized_mutual_info_score

import sys
sys.path += ['/home/lorenzo/Scrivania/My_projects/DeepWalk/P2Vec/Package/']  

from node2vec.model import Node2Vec

from p2vec import *

import warnings
warnings.filterwarnings("ignore")

directory = '/home/lorenzo/Scrivania/My_projects/DeepWalk/P2Vec/dataset/'

# Qui direi abbastanza bene. Bisogna solo fare un po' attenzione al parametro $\gamma$. `orkut` ci mette molto tempo e per `Node2Vec` non l'ho nemmeno testato.

In [4]:
# The datasets are ['amazon', 'dblp', 'livejournal', 'orkut', 'youtube']

name = 'youtube'

EL = pd.read_csv(directory + name + '.csv')[['id1', 'id2']]
n = len(pd.concat([EL.id1, EL.id2]).unique())

ℓtrue = pd.read_csv(directory + name + '_label.csv').set_index('node')
ℓtrue = ℓtrue.loc[np.arange(n)].label.values
n_clusters = len(np.unique(ℓtrue))


print("Number of nodes: " + str(n))
print("Number of clusters: " + str(n_clusters))


A = csr_matrix((np.ones(len(EL)), (EL.id1, EL.id2)), shape = (n,n))

Number of nodes: 26931
Number of clusters: 608


In [7]:
# Our method

t0 = time.time()

d = A@np.ones(n)
D_1 = diags(d**(-1))
P = D_1.dot(A)

Pv = [P]
dim = 128
k = 1
walk_length = 5
γ = 0.3

p2v = CreateEmbedding(Pv, dim = dim, walk_length = walk_length, k = k, γ = γ) 
print("\nExecution time: " + str(time.time() - t0) + "\n\n")
 
# run Kmeans
Φ = p2v.Φ
n_clusters = n_clusters

kmeans = faiss.Kmeans(Φ.shape[1], n_clusters, verbose = True)
kmeans.train(np.ascontiguousarray(Φ).astype('float32'))
_, ℓ = kmeans.assign(np.ascontiguousarray(Φ).astype('float32'))

print("\n------------------------------")
print("\nThe NMI is: " + str(normalized_mutual_info_score(ℓ, ℓtrue)))

Running the optimization for k = 1
[========================>] 100%, η = 0.001968

Computing the parameters values...

Execution time: 18.45024037361145



Clustering 26931 points in 128D to 608 clusters, redo 1 times, 25 iterations
  Preprocessing in 0.00 s
  Iteration 24 (8.01 s, search 7.73 s): objective=2097.28 imbalance=2.725 nsplit=0       
------------------------------

The NMI is: 0.6456827401351222


In [6]:
# Node2Vec

t0 = time.time()
node2vec_model = Node2Vec(EL.id1, EL.id2, graph_is_directed = False)
node2vec_model.simulate_walks()
node2vec_model.learn_embeddings(dimensions = dim, context_size = walk_length)
Y = node2vec_model.embeddings
print("\n\nExecution time: " + str(time.time() - t0) + "\n")

# run kmeans
kmeans = faiss.Kmeans(Y.shape[1], n_clusters, verbose = True)
kmeans.train(np.ascontiguousarray(Y).astype('float32'))
_, ℓDW = kmeans.assign(np.ascontiguousarray(Y).astype('float32'))

print("\n--------------------------------")
print("\nThe NMI is: " + str(normalized_mutual_info_score(ℓDW, ℓtrue)))


Preprocessing progress: 0%Learning embeddings using the Skip-Gram model...
Preprocessing progress: 99.14%95.8%ng progress: 98.498.4%%%%
Preprocessing transition probabilities: done!
Walking progress: 96.54%g progress: 0%0%0%g progress: 
Simulating random walks: done!
Learning embeddings: done!


Execution time: 59.04410696029663

Clustering 26931 points in 128D to 608 clusters, redo 1 times, 25 iterations
  Preprocessing in 0.00 s
  Iteration 24 (8.28 s, search 8.01 s): objective=82601.3 imbalance=2.023 nsplit=0       
--------------------------------

The NMI is: 0.6446431446311961
